In [1]:
import math
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss, classification_report
import torchvision
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, SubsetRandomSampler, Subset
from torchvision import transforms as T
from collections import namedtuple
from itertools import product
import time
from collections import OrderedDict
import json
from IPython.display import clear_output

In [33]:

class Bottleneck(nn.Module):
    def __init__(self, in_planes, growth_rate):
        super(Bottleneck, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv1 = nn.Conv2d(in_planes, 4*growth_rate, kernel_size=1, bias=False)
        self.bn2 = nn.BatchNorm2d(4*growth_rate)
        self.conv2 = nn.Conv2d(4*growth_rate, growth_rate, kernel_size=3, padding=1, bias=False)

    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = self.conv2(F.relu(self.bn2(out)))
        out = torch.cat([out,x], 1)
        return out


class Transition(nn.Module):
    def __init__(self, in_planes, out_planes):
        super(Transition, self).__init__()
        self.bn = nn.BatchNorm2d(in_planes)
        self.conv = nn.Conv2d(in_planes, out_planes, kernel_size=1, bias=False)

    def forward(self, x):
        out = self.conv(F.relu(self.bn(x)))
        out = F.avg_pool2d(out, 2)
        return out


class DenseNet(nn.Module):
    def __init__(self, block, nblocks, growth_rate=12, reduction=0.5, num_classes=100):
        super(DenseNet, self).__init__()
        self.growth_rate = growth_rate

        num_planes = 2*growth_rate
        self.conv1 = nn.Conv2d(3, num_planes, kernel_size=3, padding=1, bias=False)

        self.dense1 = self._make_dense_layers(block, num_planes, nblocks[0])
        num_planes += nblocks[0]*growth_rate
        out_planes = int(math.floor(num_planes*reduction))
        self.trans1 = Transition(num_planes, out_planes)
        num_planes = out_planes

        self.dense2 = self._make_dense_layers(block, num_planes, nblocks[1])
        num_planes += nblocks[1]*growth_rate
        out_planes = int(math.floor(num_planes*reduction))
        self.trans2 = Transition(num_planes, out_planes)
        num_planes = out_planes

        self.dense3 = self._make_dense_layers(block, num_planes, nblocks[2])
        num_planes += nblocks[2]*growth_rate
        out_planes = int(math.floor(num_planes*reduction))
        self.trans3 = Transition(num_planes, out_planes)
        num_planes = out_planes

        self.dense4 = self._make_dense_layers(block, num_planes, nblocks[3])
        num_planes += nblocks[3]*growth_rate

        self.bn = nn.BatchNorm2d(num_planes)
        self.linear = nn.Linear(256, 100)

    def _make_dense_layers(self, block, in_planes, nblock):
        layers = []
        for i in range(nblock):
            layers.append(block(in_planes, self.growth_rate))
            in_planes += self.growth_rate
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.trans1(self.dense1(out))
        out = self.trans2(self.dense2(out))
        out = self.trans3(self.dense3(out))
        out = self.dense4(out)
        out = F.avg_pool2d(F.relu(self.bn(out)), 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def DenseNet121():
    return DenseNet(Bottleneck, [6,12,24,16], growth_rate=32)

def DenseNet169():
    return DenseNet(Bottleneck, [6,12,32,32], growth_rate=32)

def DenseNet201():
    return DenseNet(Bottleneck, [6,12,48,32], growth_rate=32)

def DenseNet161():
    return DenseNet(Bottleneck, [6,12,36,24], growth_rate=48)

def densenet_cifar():
    return DenseNet(Bottleneck, [6,12,24,16], growth_rate=8)

def test():
    net = densenet_cifar()
    x = torch.randn(256,3,32,32)
    y = net(x)
    print(y)

In [34]:
test()

tensor([[ 0.1485, -0.5054, -0.2930,  ..., -0.2387, -0.2448, -0.2375],
        [ 0.3814, -0.6060, -0.1572,  ..., -0.2181, -0.1509, -0.0927],
        [ 0.2260, -0.5723, -0.2640,  ..., -0.0791, -0.2204, -0.2124],
        ...,
        [ 0.1761, -0.6729, -0.1266,  ..., -0.0756, -0.2956, -0.2218],
        [ 0.3510, -0.4607, -0.3269,  ..., -0.2043, -0.1229, -0.2052],
        [ 0.1784, -0.6587, -0.2755,  ..., -0.1956, -0.2263, -0.0527]],
       grad_fn=<AddmmBackward0>)


In [2]:
M_full = pd.read_csv('./final_datasets/M_train.csv')
M_solution = pd.read_csv('./final_datasets/M_solution.csv')
W_full = pd.read_csv('./final_datasets/W_train.csv')
W_solution = pd.read_csv('./final_datasets/W_solution.csv')

In [3]:
M_test = M_full[M_full['Season'] >= 2018]
M_train = M_full[M_full['Season'] < 2018]

In [4]:
X_train_M = M_train.drop(['Season','T1_TeamID','T2_TeamID','win'], axis=1)
X_test_M = M_test.drop(['Season','T1_TeamID','T2_TeamID','win'], axis=1)
y_train_M = M_train['win']
y_test_M = M_test['win']

In [5]:
def get_default_device():
  if torch.cuda.is_available():
    return torch.device('cuda')
  return torch.device('cpu')


device=get_default_device()
device

device(type='cpu')

In [57]:
train_loader = torch.utils.data.DataLoader(
      M_train,
      batch_size=32,
      num_workers=4,shuffle = True,
)
test_loader = torch.utils.data.DataLoader(
    M_test,
    batch_size=32,
    shuffle=False,
    num_workers=4,
)
lr = 0.01
model = DenseNet121().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.5)
loss_fn = nn.CrossEntropyLoss()

In [58]:
class RunBuilder():
  @staticmethod
  def get_runs(params):

    Run = namedtuple('Run', params.keys())

    runs = []
    for v in product(*params.values()):
      runs.append(Run(*v))

    return runs

In [62]:
import time
# Helper class, help track loss, accuracy, epoch time, run time,
# hyper-parameters etc.
class RunManager():
  def __init__(self):

    # tracking every epoch count, loss, accuracy, time
    self.epoch_count = 0
    self.epoch_loss = 0
    self.epoch_num_correct = 0
    self.epoch_start_time = None

    # tracking every run count, run data, hyper-params used, time
    self.run_params = None
    self.run_count = 0
    self.run_data = []
    self.run_start_time = None

    # record model, loader and TensorBoard
    self.network = None
    self.loader = None
    self.loader_test = None

  # record the count, hyper-param, model, loader of each run
  # record sample images and network graph to TensorBoard
  def begin_run(self, run, network, loader, loader_test):

    self.run_start_time = time.time()

    self.run_params = run
    self.run_count += 1

    self.network = network
    self.loader = loader
    self.loader_test = loader_test

  # when run ends, close TensorBoard, zero epoch count
  def end_run(self):
    self.epoch_count = 0

  # zero epoch count, loss, accuracy,
  def begin_epoch(self):
    self.epoch_start_time = time.time()

    self.epoch_count += 1
    self.epoch_loss = 0
    self.epoch_num_correct = 0

    self.epoch_loss_test = 0
    self.epoch_num_correct_test = 0
  def end_epoch(self):
    # calculate epoch duration and run duration(accumulate)
    epoch_duration = time.time() - self.epoch_start_time
    run_duration = time.time() - self.run_start_time

    # record epoch loss and accuracy
    loss = self.epoch_loss / len(self.loader.dataset)
    accuracy = self.epoch_num_correct / len(self.loader.dataset)

    accuracy_test = self.epoch_num_correct_test / len(self.loader_test.dataset)

    # Write into 'results' (OrderedDict) for all run related data
    results = OrderedDict()
    results["run"] = self.run_count
    results["epoch"] = self.epoch_count
    results["loss"] = loss
    results["accuracy"] = accuracy
    results["accuracy_test"] = accuracy_test
    results["epoch duration"] = epoch_duration
    results["run duration"] = run_duration

    # Record hyper-params into 'results'
    for k,v in self.run_params._asdict().items(): results[k] = v
    self.run_data.append(results)
    df = pd.DataFrame.from_dict(self.run_data, orient = 'columns')

    # display epoch information and show progress
    clear_output(wait=True)
    display(df)

  # accumulate loss of batch into entire epoch loss
  def track_loss(self, loss):
    # multiply batch size so variety of batch sizes can be compared
    self.epoch_loss += loss.item() * self.loader.batch_size
  # accumulate number of corrects of batch into entire epoch num_correct
  def track_num_correct(self, preds, labels):
    self.epoch_num_correct += self._get_num_correct(preds, labels)

  def track_num_correct_test(self, preds, labels):
    self.epoch_num_correct_test += self._get_num_correct(preds, labels)

  @torch.no_grad()
  def _get_num_correct(self, preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [63]:
epochs = 10
def searching_function(params2, return_model=False, network = None):
  m = RunManager()

  # get all runs from params using RunBuilder class
  for run in RunBuilder.get_runs(params2):

    # if params changes, following line of code should reflect the changes too
    if network is None:
      network = torchvision.models.densenet121()
      for param in network.parameters():
        param.required_grad = False
      network.classifier = nn.Linear(1024, 100)
    network.to(device)
    loader_test = torch.utils.data.DataLoader(M_test, batch_size = run.batch_size,shuffle=True,num_workers=4)
    loader = torch.utils.data.DataLoader(M_train, batch_size = run.batch_size,shuffle=True,num_workers=4)
    optimizer = torch.optim.SGD(network.parameters(), lr=run.lr, momentum=0.5)

    m.begin_run(run, network, loader, loader_test)
    for epoch in range(epochs):
      m.begin_epoch()

      for batch in loader_test:
        print(batch)
        data = batch.to(device)
        score = batch['win'].to(device)
        preds = network(data)
        m.track_num_correct_test(preds, score)
        del data, score, preds
        gc.collect()
        torch.cuda.empty_cache()


      for batch in loader:

        data = batch[0].to(device)
        score = batch['win'].to(device)
        preds = network(data).to(device)
        loss = F.cross_entropy(preds, score.long())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        m.track_loss(loss)
        m.track_num_correct(preds, score)
        del data, labels, preds
        gc.collect()
        torch.cuda.empty_cache()

      m.end_epoch()
    m.end_run()

    # when all runs are done, save results to files
    if return_model:
      return network

In [64]:
from collections import OrderedDict, namedtuple
from itertools import product
params = OrderedDict(
    lr = [0.01],
    batch_size = [16],
    shuffle = [True],
)
trained_model = searching_function(params,return_model=True)

KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "C:\Users\alekm\.conda\envs\March-Machine-Learning-Mania-2023\lib\site-packages\pandas\core\indexes\base.py", line 3802, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 165, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 5745, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 5753, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 279

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\alekm\.conda\envs\March-Machine-Learning-Mania-2023\lib\site-packages\torch\utils\data\_utils\worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "C:\Users

In [ ]:
params = OrderedDict(
    lr = [0.01],
    batch_size = [16],
    shuffle = [True],
)
trained_modelv2 = searching_function(params,return_model=True,network=trained_model)

# Alternative

In [6]:
import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision
from torch.utils.data import DataLoader,Subset
import torch.nn as nn

In [7]:
def evaluation(dataloader, model):
    total, correct = 0, 0
    for data in dataloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, pred = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (pred == labels).sum().item()
    return 100 * correct / total

In [8]:
M_train.columns

Index(['Season', 'T1_TeamID', 'T1_FG%_home_mean', 'T1_3P%_home_mean',
       'T1_point_diff_home_mean', 'T1_win_home_mean', 'T1_FG%_away_mean',
       'T1_3P%_away_mean', 'T1_point_diff_away_mean', 'T1_win_away_mean',
       ...
       'T2_top8_appearances', 'T2_top16_appearances', 'T2_top32_appearances',
       'T2_top64_appearances', 'T2_top68_appearances',
       'T2_total_tourney_appearances', 'T2_conf_point_margin',
       'T2_conf_win_margin', 'win', 'seed_diff'],
      dtype='object', length=121)

In [28]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch import tensor
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18
from sklearn.model_selection import train_test_split

class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = tensor(self.data.iloc[index].values, dtype=torch.float32)
        y = tensor(self.labels.iloc[index], dtype=torch.float32)
        return x, y

In [34]:
import gc
batch_size = 16
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
iterations = []
accuracies = []
losses = []
M_full = pd.read_csv('./final_datasets/M_train.csv')
M_solution = pd.read_csv('./final_datasets/M_solution.csv')
M_test = M_full[M_full['Season'] >= 2018]
M_train = M_full[M_full['Season'] < 2018]
X_train_M = M_train.drop(['Season', 'T1_TeamID', 'T2_TeamID', 'win'], axis=1)
X_test_M = M_test.drop(['Season', 'T1_TeamID', 'T2_TeamID', 'win'], axis=1)
y_train_M = M_train['win']
y_test_M = M_test['win']
train_data = CustomDataset(X_train_M, y_train_M)
test_data = CustomDataset(X_test_M, y_test_M)
trainloader = DataLoader(train_data, batch_size = batch_size, shuffle =  True)
testloader = DataLoader(test_data, batch_size = batch_size, shuffle = True)
model = torchvision.models.resnet18(pretrained=True)
model.fc = nn.Linear(X_train_M.shape[1], 2)
model = model.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
loss_epoch_arr = []
max_epochs = 25

min_loss = 1000000

n_iters = np.ceil(10436/batch_size)*max_epochs
iters = 0

for epoch in range(max_epochs):
    model.train()
    for data, target in trainloader:
        iters += 1

        opt.zero_grad()

        outputs = model(data)
        loss = loss_fn(outputs, target)
        loss.backward()
        opt.step()
        del data, target, outputs

        gc.collect()
        torch.cuda.empty_cache()
    loss_epoch_arr.append(loss.item())
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += loss_fn(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    acc = 100. * correct / len(test_loader.dataset)
    print('Epoch: %d/%d ended. Train acc: %0.2f' % (epoch+1, max_epochs, acc))

print('\n\nTest Accuracy on final model: %0.4f' % evaluation(testloader, model))

C:\Users\alekm\.conda\envs\March-Machine-Learning-Mania-2023\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
C:\Users\alekm\.conda\envs\March-Machine-Learning-Mania-2023\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [16, 117]